In [1]:
from mhr.utils.utils import load_json_file, process_jsonl,write_jsonl
from py2neo import *
from neo4j import GraphDatabase
from tqdm import tqdm

In [2]:
pope_result_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/robustlmm/dr_algo/0601_2205_eval_all_llava_baseline/pope.json"
pope_result_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0312_1719_eval_pope_mul_baseline/pope_mul_en.json"
pope_object_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/pope/pope_llama_objects.jsonl"
output_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/pope/pope_object_rank.jsonl"

pope_aokvqa_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1558_eval_pope_aokvqa_mul_baseline/pope_aokvqa_mul_en.json"
pope_coco_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1558_eval_pope_mul_baseline/pope_mul_en.json"
pope_gqa_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1559_eval_pope_gqa_mul_baseline/pope_gqa_mul_en.json"
pope_result_file = [pope_aokvqa_file, pope_coco_file, pope_gqa_file]

def get_results_from_result_file(file):
    if isinstance(file, str):
        result = load_json_file(file)["logs"]
    elif isinstance(file, list):
        result = []
        for f in file:
            result.extend(load_json_file(f)["logs"])
    else:
        raise ValueError("file should be a string or a list of string")
    return result

def from_str_to_list(string):
    return string.strip().replace("[", "").replace("]", "").split(",")

def get_object_dict(object_file):
    object_data = process_jsonl(object_file)
    object_dict = {}
    for obj in object_data:
        object_dict[obj["id"]] = list(set(from_str_to_list(obj["objects"])))
    return object_dict

def get_object_statistic_by_neo4j(database="instructmix665kllama"):
    uri = "neo4j://10.140.54.16:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
    query = "match (n) return n;"
    with driver.session(database=database) as session:
        results = session.run(query).data()
        results.sort(key=lambda x: x['n']['count'], reverse=True)
        
    object_statistic_dict = {}
    for item in results:
        object_statistic_dict[item['n']['lemma']] = dict(num=item['n']['count'],rank=len(object_statistic_dict)+1)
    return object_statistic_dict 

In [3]:
object_dict = get_object_dict(pope_object_file)
print(object_dict)
object_statistic_dict = get_object_statistic_by_neo4j("instructmix665kllama")
result_list = get_results_from_result_file(pope_result_file)
analyze_results = []

wrong_max_ranks = []
correct_max_ranks = []
wrong_mean_ranks = []
correct_mean_ranks = []
wrong_min_ranks = []
correct_min_ranks = []

wrong_max_counts = []
correct_max_counts = []
wrong_mean_counts = []
correct_mean_counts = []
wrong_min_counts = []
correct_min_counts = []

for log in tqdm(result_list):
    id = log["doc"]["question_id"]
    image_key = log["doc"]["image"]
    correct = log["pope_accuracy"]["score"] > 0.0
    now_object_list = object_dict.get(image_key,None)
    if now_object_list is None:
        continue
    current_num = []
    current_rank = []
    for object in now_object_list:
        if object_statistic_dict.get(object, None) is None:
            continue
        current_num.append(object_statistic_dict[object]["num"])
        current_rank.append(object_statistic_dict[object]["rank"])
    max_rank = max(current_rank)
    min_rank = min(current_rank)
    max_num = max(current_num)
    min_num = min(current_num)
    mean_rank = sum(current_rank) / len(current_rank)
    mean_num = sum(current_num) / len(current_num)
    if correct:
        correct_max_ranks.append(max_rank)
        correct_min_ranks.append(min_rank)
        correct_mean_ranks.append(mean_rank)
        correct_max_counts.append(max_num)
        correct_min_counts.append(min_num)
        correct_mean_counts.append(mean_num)
    else:
        wrong_max_ranks.append(max_rank)
        wrong_min_ranks.append(min_rank)
        wrong_mean_ranks.append(mean_rank)
        wrong_max_counts.append(max_num)
        wrong_min_counts.append(min_num)
        wrong_mean_counts.append(mean_num)
    
    
    analyze_results.append(dict(id=id, correct=correct, 
                                max_rank=max_rank, min_rank=min_rank, mean_rank=mean_rank, max_num=max_num, min_num=min_num, mean_num=mean_num,
                                rank_list=current_rank, num_list=current_num))
print("correct max rank:", sum(correct_max_ranks) / len(correct_max_ranks))
print("correct min rank:", sum(correct_min_ranks) / len(correct_min_ranks))
print("correct mean rank:", sum(correct_mean_ranks) / len(correct_mean_ranks))
print("wrong max rank:", sum(wrong_max_ranks) / len(wrong_max_ranks))
print("wrong min rank:", sum(wrong_min_ranks) / len(wrong_min_ranks))
print("wrong mean rank:", sum(wrong_mean_ranks) / len(wrong_mean_ranks))
print("correct max count:", sum(correct_max_counts) / len(correct_max_counts))
print("correct min count:", sum(correct_min_counts) / len(correct_min_counts))
print("correct mean count:", sum(correct_mean_counts) / len(correct_mean_counts))
print("wrong max count:", sum(wrong_max_counts) / len(wrong_max_counts))
print("wrong min count:", sum(wrong_min_counts) / len(wrong_min_counts))
print("wrong mean count:", sum(wrong_mean_counts) / len(wrong_mean_counts))
write_jsonl(analyze_results,output_file)
        



{'COCO_val2014_000000461751.jpg': ['back', 'car', 'street', 'man', 'motorcycle', 'helmet'], 'COCO_val2014_000000377368.jpg': ['cake', 'candles', 'people', 'teddy bear', 'table', 'c'], 'COCO_val2014_000000563603.jpg': ['field', 'giraffes', 'tree'], 'COCO_val2014_000000329542.jpg': ['b', 'cupcake', 'cupcakes', 'cherry', 'hand', 'tray'], 'COCO_val2014_000000182202.jpg': ['cell phone', 'laptop', 'desk'], 'COCO_val2014_000000198960.jpg': ['field', 'sheep', 'fence'], 'COCO_val2014_000000257169.jpg': ['sink', 'bottles', 'person', 'towels', 'hair dryer'], 'COCO_val2014_000000004795.jpg': ['computer', 'keyboard', 'cat', 'monitor', 'cell phone'], 'COCO_val2014_000000163640.jpg': ['sink', 'room', 'men', 'donut', 'floor', 'window'], 'COCO_val2014_000000071711.jpg': ['airplanes', 'hangar', 'people'], 'COCO_val2014_000000117425.jpg': ['cake', 'chairs', 'candle', 'knife', 'table', 'child'], 'COCO_val2014_000000250766.jpg': ['meat', 'vegetables', 'bro', 'plate', 'salad'], 'COCO_val2014_000000147415.jp

100%|██████████| 9000/9000 [00:00<00:00, 173346.20it/s]

correct max rank: 1085.0681994316715
correct min rank: 17.975587703435806
correct mean rank: 319.927935441808
wrong max rank: 1100.7297297297298
wrong min rank: 21.04054054054054
wrong mean rank: 336.3830305095012
correct max count: 31623.859597003357
correct min count: 1117.9475587703437
correct mean count: 10392.318610916416
wrong max count: 31153.964228934816
wrong min count: 1044.3290937996821
wrong mean count: 10185.244117647053


In [4]:
object_dict = get_object_dict(pope_object_file)
print(object_dict)
object_statistic_dict = get_object_statistic_by_neo4j("instructmix665kllama")
result_list = load_json_file(pope_result_file)["logs"]
analyze_results = []
wrong_max_ranks = []
correct_max_ranks = []
wrong_mean_ranks = []
correct_mean_ranks = []
wrong_min_ranks = []
correct_min_ranks = []
for log in tqdm(result_list):
    id = log["doc"]["question_id"]
    image_key = log["doc"]["image"]
    correct = log["pope_accuracy"]["score"] > 0.0
    now_object_list = object_dict.get(image_key,None)
    if now_object_list is None:
        continue
    current_num = []
    current_rank = []
    for object in now_object_list:
        if object_statistic_dict.get(object, None) is None:
            continue
        current_num.append(object_statistic_dict[object]["num"])
        current_rank.append(object_statistic_dict[object]["rank"])
    max_rank = max(current_rank)
    min_rank = min(current_rank)
    max_num = max(current_num)
    min_num = min(current_num)
    mean_rank = sum(current_rank) / len(current_rank)
    mean_num = sum(current_num) / len(current_num)
    if correct:
        correct_max_ranks.append(max_rank)
        correct_min_ranks.append(min_rank)
        correct_mean_ranks.append(mean_rank)
    else:
        wrong_max_ranks.append(max_rank)
        wrong_min_ranks.append(min_rank)
        wrong_mean_ranks.append(mean_rank)
    
    
    analyze_results.append(dict(id=id, correct=correct, 
                                max_rank=max_rank, min_rank=min_rank, mean_rank=mean_rank, max_num=max_num, min_num=min_num, mean_num=mean_num,
                                rank_list=current_rank, num_list=current_num))
print("correct max rank:", sum(correct_max_ranks) / len(correct_max_ranks))
print("correct min rank:", sum(correct_min_ranks) / len(correct_min_ranks))
print("correct mean rank:", sum(correct_mean_ranks) / len(correct_mean_ranks))
print("wrong max rank:", sum(wrong_max_ranks) / len(wrong_max_ranks))
print("wrong min rank:", sum(wrong_min_ranks) / len(wrong_min_ranks))
print("wrong mean rank:", sum(wrong_mean_ranks) / len(wrong_mean_ranks))
write_jsonl(analyze_results,output_file)
        



{'COCO_val2014_000000461751.jpg': ['back', 'car', 'street', 'man', 'motorcycle', 'helmet'], 'COCO_val2014_000000377368.jpg': ['cake', 'candles', 'people', 'teddy bear', 'table', 'c'], 'COCO_val2014_000000563603.jpg': ['field', 'giraffes', 'tree'], 'COCO_val2014_000000329542.jpg': ['b', 'cupcake', 'cupcakes', 'cherry', 'hand', 'tray'], 'COCO_val2014_000000182202.jpg': ['cell phone', 'laptop', 'desk'], 'COCO_val2014_000000198960.jpg': ['field', 'sheep', 'fence'], 'COCO_val2014_000000257169.jpg': ['sink', 'bottles', 'person', 'towels', 'hair dryer'], 'COCO_val2014_000000004795.jpg': ['computer', 'keyboard', 'cat', 'monitor', 'cell phone'], 'COCO_val2014_000000163640.jpg': ['sink', 'room', 'men', 'donut', 'floor', 'window'], 'COCO_val2014_000000071711.jpg': ['airplanes', 'hangar', 'people'], 'COCO_val2014_000000117425.jpg': ['cake', 'chairs', 'candle', 'knife', 'table', 'child'], 'COCO_val2014_000000250766.jpg': ['meat', 'vegetables', 'bro', 'plate', 'salad'], 'COCO_val2014_000000147415.jp

TypeError: expected str, bytes or os.PathLike object, not list